# Quantum Computing Image Recognition

In [1]:
import numpy as np

# Load training data and test data
train_dataset = "./dataset/recycled_32_train.npz"
train_data = np.load(train_dataset)

test_dataset = "./dataset/recycled_32_test.npz"
test_data = np.load(test_dataset)

The "x" of the dataset contains a two dimension numpy array of uint8, where each
row contains a 32x32 coloured image. The picture follows the "channel first" rule.

The "y" of the dataset contains a one dimension numpy array of uint8, where each
value indicates the label of corresponding x item.


In [2]:
# Train data
train_images = train_data["x"]
train_labels = train_data["y"]

# Test data
test_images = test_data["x"]
test_labels = test_data["y"]

In [3]:
# Normalize the data
train_images = train_images / 255.0
test_images = test_images / 255.0

# Reshape the data
train_images = train_images.reshape(train_images.shape[0], 32, 32, 3)

test_images = test_images.reshape(test_images.shape[0], 32, 32, 3)

In [5]:
# Build the model using Quantum Convolutional Neural Network

import tensorflow as tf
import tensorflow_quantum as tfq

# Define the qubits and readout operators
qubits = cirq.GridQubit.rect(4, 4)
readout_operators = [cirq.Z(qubits[-1])]
inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string)
circuit_input = tfq.layers.AddCircuit()([inputs, tfq.convert_to_tensor([circuit])])
expectation_layer = tfq.layers.Expectation()
expectations = expectation_layer(circuit_input)
model = tf.keras.Model(inputs=inputs, outputs=expectations)

# Define the classical neural network
cnn_model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.MaxPooling2D((2, 2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Define the quantum neural network
quantum_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(4, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tfq.layers.PQC(circuit, readout_operators)
])

# Define the hybrid model
combined_model = tf.keras.Sequential([
    tf.keras.layers.Input(shape=(32, 32, 3)),
    tf.keras.layers.Conv2D(4, (3, 3), activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10)
])

# Compile the model
cnn_model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

quantum_model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])

combined_model.compile(optimizer='adam',
                    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                    metrics=['accuracy'])


ModuleNotFoundError: No module named 'tensorflow_quantum'

In [4]:
# Train the classical model
cnn_model.fit(train_images, train_labels, epochs=10,
                validation_data=(test_images, test_labels))

NameError: name 'cnn_model' is not defined

In [ ]:
# Train the quantum model
quantum_model.fit(train_images, train_labels, epochs=10,
                validation_data=(test_images, test_labels))

In [ ]:
# Train the hybrid model
combined_model.fit(train_images, train_labels, epochs=10,
                validation_data=(test_images, test_labels))

In [ ]:
# Evaluate the classical model
test_loss, test_acc = cnn_model.evaluate(test_images,  test_labels, verbose=2)

# Evaluate the quantum model

test_loss, test_acc = quantum_model.evaluate(test_images,  test_labels, verbose=2)

# Evaluate the hybrid model

test_loss, test_acc = combined_model.evaluate(test_images,  test_labels, verbose=2)

# Save the model
cnn_model.save("cnn_model.h5")
quantum_model.save("quantum_model.h5")
combined_model.save("combined_model.h5")

In [ ]:
# Load the model
cnn_model = tf.keras.models.load_model("cnn_model.h5", custom_objects={'circuit': circuit})
quantum_model = tf.keras.models.load_model("quantum_model.h5", custom_objects={'circuit': circuit})
combined_model = tf.keras.models.load_model("combined_model.h5", custom_objects={'circuit': circuit})

# Make predictions
predictions = cnn_model.predict(test_images)
predictions = quantum_model.predict(test_images)
predictions = combined_model.predict(test_images)

# Plot the predictions
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 10))
for i in range(9):
    plt.subplot(3, 3, i + 1)
    plt.xticks([])
    plt.yticks([])
    plt.grid(False)
    plt.imshow(test_images[i], cmap=plt.cm.binary)
    plt.xlabel(np.argmax(predictions[i]))
plt.show()



In [ ]:
# Measure the accuracy

from sklearn.metrics import accuracy_score

y_pred = np.argmax(predictions, axis=1)

# Measure the accuracy of the classical model
predictions = cnn_model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)
cnn_score = accuracy_score(test_labels, y_pred)


# Measure the accuracy of the quantum model
predictions = quantum_model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)
quantum_score = accuracy_score(test_labels, y_pred)

# Measure the accuracy of the hybrid model
predictions = combined_model.predict(test_images)
y_pred = np.argmax(predictions, axis=1)
hybrid_score = accuracy_score(test_labels, y_pred)

In [ ]:
# Measure the speed of the classical model
import time

start = time.time()
predictions = cnn_model.predict(test_images)
end = time.time()
print(end - start)
# Save speed of classical model
cnn_speed = end - start

# Measure the speed of the quantum model
start = time.time()
predictions = quantum_model.predict(test_images)
end = time.time()
print(end - start)
# Save speed of quantum model
quantum_speed = end - start

# Measure the speed of the hybrid model
start = time.time()
predictions = combined_model.predict(test_images)
end = time.time()
print(end - start)
# Save speed of hybrid model
hybrid_speed = end - start

# Make a graph of the differences in accuracy and speed
import matplotlib.pyplot as plt

accuracy = [cnn_score, quantum_score, hybrid_score]
speed = [cnn_speed, quantum_speed, hybrid_speed]
labels = ["Classical", "Quantum", "Hybrid"]

x = np.arange(len(labels))
width = 0.35

fig, ax = plt.subplots()

rects1 = ax.bar(x - width/2, accuracy, width, label='Accuracy')
rects2 = ax.bar(x + width/2, speed, width, label='Speed')

ax.set_ylabel('Accuracy/Speed')
ax.set_title('Accuracy and Speed of Classical, Quantum, and Hybrid Models')
ax.set_xticks(x)
ax.set_xticklabels(labels)
ax.legend()

fig.tight_layout()

plt.show()